In [1]:
from bs4 import BeautifulSoup
import urllib.request as urllib_request
from urllib.request import urlopen
import pandas as pd
import numpy  as np

# funções

In [2]:
def webscrapping(url):
  response = urlopen(url)
  html = response.read()
  soup = BeautifulSoup(html, 'html.parser')
  links = []
  for item in soup.findAll('a', {"class": "resource-url-analytics"}):
    links.append(item.get('href'))
  df_final = pd.DataFrame()
  for i in range(2,18):
    df = pd.read_csv(links[i], encoding='ISO8859',skiprows=7, on_bad_lines='skip', sep=';')
    df = df.iloc[1:28]
    df['CSV Nome'] = links[i].split('/')[9].split('.')[0]
      
    df_final = pd.concat([df_final,df])
  return df_final

In [3]:
url = 'https://www.tesourotransparente.gov.br/ckan/dataset/transferencias-obrigatorias-da-uniao'
df = webscrapping(url)

In [5]:
df.to_csv('dados_brutos.csv')

In [6]:
def tratamento_dados(df_final,estado,ano_inicial,ano_final):
  df_final['ESTADOS'].fillna(df_final['Capitais'], inplace=True)
  df_final = df_final.replace(' -   ', '0')
  dft = df_final[(df_final['ESTADOS'] == estado) ]\
  .melt(id_vars=['CSV Nome', 'ESTADOS','UF'], 
      value_vars = [col for col in df_final.columns 
                    if col not in ['CSV Nome', 'ESTADOS','UF']])
  mes_numbers = {'janeiro':1,'fevereiro':2,'março':3,'abril':4,'maio':5,
                'junho':6,'julho':7,'agosto':8,'setembro':9,'outubro':10,
                'novembro':11,'dezembro':12}

  init_years_range = ano_inicial
  final_years_range = ano_final

  df_all = ''

  for year in range(init_years_range,final_years_range+1):
    cols = ['CSV Nome', 'ESTADOS','UF'] + [col for col in df_final.columns if 
                                          col.endswith(f'/{year}')]
    dft  = df_final.loc[:,cols]

    dft = dft.melt(id_vars=['CSV Nome', 'ESTADOS','UF'], 
          value_vars = [col for col in dft.columns 
                        if col not in ['CSV Nome', 'ESTADOS','UF']])

    dft[['Mês','Ano']] = dft['variable'].str.split('/',expand=True)
    
    dft.drop(['variable','Ano'],inplace=True,axis=1)
    dft.rename(columns={'value':f'20{year}'},inplace=True)
    dft['Mês'] = dft['Mês'].replace(mes_numbers)
    dft = dft[['CSV Nome', 'ESTADOS','UF','Mês', f'20{year}']]
    
    if isinstance(df_all,str) == True:
      df_all = dft
    else:
      df_all = df_all.merge(dft,on=['CSV Nome','ESTADOS','Mês','UF'],how='left')
    df_all.fillna(0)
  return df_all[df_all['ESTADOS'] == estado]

In [8]:
df2 = tratamento_dados(df,estado='Alagoas', ano_inicial=17, ano_final=22)
df2

,CSV Nome,ESTADOS,UF,Mês,2017,2018,2019,2020,2021,2022
1,CIDEEST,Alagoas,AL,1,"4.492.196,43","4.485.114,67","2.295.681,62","2.141.474,89","966.410,06","2.230.921,30"
28,CIDEMUN,Alagoas,AL,1,"1.497.398,81","1.495.038,22","765.227,21","713.824,96","322.136,69","743.640,43"
55,FEXEST,Alagoas,AL,1,0,0,0,0,0,0
82,FEXMUN,Alagoas,AL,1,0,0,0,0,0,0
109,FPE,Alagoas,AL,1,"242.256.259,13","256.938.067,84","316.145.617,39","284.004.528,95","343.522.250,88","425.107.461,45"
...,...,...,...,...,...,...,...,...,...,...
5050,LC87EST,Alagoas,AL,12,"819.214,51","802.410,11",0,0,0,0
5077,LC87MUN,Alagoas,AL,12,"273.071,89","267.470,41",0,0,0,0
5104,LC176EST,Alagoas,AL,12,NaN,NaN,NaN,"13.501.500,00","1.374.900,06",0
5131,LC176MUN,Alagoas,AL,12,NaN,NaN,NaN,"2.193.476,20","452.731,20",0


In [10]:
df2.to_csv('dados_tratados_alagoas.csv')